In [ ]:
def calc_G(F,gamman,b,V,gn_edges):
    Gint = xr.DataArray(np.zeros(shape=(gn_edges.size,F.time.size)),coords={'gn': gn_edges,'time': F.time},dims=['gn','time'])
    gFbV = gamman*b*F*V
    i=0
    for g in gn_edges:
        mask = gamman<g
        Gint.loc[dict(gn=g)] = gFbV.where(mask,0).sum(dim=['lat','lon','depth'])
        i+=1
    G = Gint.differentiate('gn')
    
    return G

In [ ]:
# Alternative, discrete volume calculation derived in Appendix 7.5 of Groeskamp et al (2018)
def calc_G_alt(F,gamman,b,V,gn_edges):
    gn_centres = (gn_edges[1:] + gn_edges[:-1]) / 2
    dgn = np.diff(gn_edges)
    G = xr.DataArray(np.zeros(shape=(gn_centres.size,F.time.size)),coords={'gn': gn_centres,'time': F.time},dims=['gn','time'])
    gFbV = gamman*b*F*V
    for i in range(gn_edges.size-1):
        mask = (gamman>gn_edges[i]) & (gamman<gn_edges[i+1])
        G.loc[dict(gn=gn_centres[i])] = gFbV.where(mask,0).sum(dim=['lat','lon','depth'],skipna=True)/dgn[i]
    
    return G

In [ ]:
Gclim_vals=np.zeros(shape=(G.gamman_bin.size,12))
for i in range(12):
    indices = np.arange(i,np.diff(time_bnds),12)
    Gclim_vals[:,i] = G.isel(time=indices).mean(dim='time')
Gclim=xr.DataArray(np.tile(Gclim_vals,int(np.diff(time_bnds)/12)),coords={'gamman_bin':G.gamman_bin,'time':G.time},dims={'gamman_bin','time'})
Gannualanom = (G-Gclim).coarsen(time=12).mean()
Gannualanom.transpose().plot()